In [1]:
import copy
import this
code_word_length = 24
message = list([84, 175, 25, 72, 44, 149, 2, 31, 154, 201, 102, 145, 25, 76, 76, 51, 13, 230, 162, 80, 13, 156, 118, 9, 141, 77, 105, 177, 9, 78, 30, 129, 138, 31, 96, 16])

string = str()
for i in range(len(message)):
    string += str(bin(message[i])[2:].zfill(8)) # делаем бинарное преобразование

parts=[]
con = len(string) % code_word_length
if con != 0:
    con = 1

for i in range(len(string) // code_word_length + con): # разбиваем на кодовые слова по 24 бит
    parts.append(string[(code_word_length*i):(code_word_length*(i+1))])
reversed_parts = copy.deepcopy(parts)
parts[-1] = parts[-1].ljust(code_word_length, '0')
for i in range(len(reversed_parts)):
    reversed_parts[i] = reversed_parts[i][::-1]
reversed_parts[-1] = reversed_parts[-1].ljust(code_word_length, '0')
print(parts)
print(reversed_parts) # но в итоге ещё в каждом кодовом реверс делаем

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
['010101001010111100011001', '010010000010110010010101', '000000100001111110011010', '110010010110011010010001', '000110010100110001001100', '0

In [177]:
bits = [0,1,3,7,15]
controlled_bits = dict()

In [178]:
for k in range(5):
    n = bits[k]
    controlled_bits[n] = list()
    for i in range(n, code_word_length, (n+1)*2):
        for j in range ((n+1)*2):
            ind = i + j
            if j < (n+1) and ind < code_word_length:
                controlled_bits[n].append(ind)

In [179]:
def count_control_sum(n, parts): # вычисляем контрольную сумму
    control_sum = list()
    for i in range(5):
        m = bits[i]
        control_sum.append(0)
        for j in range(len(controlled_bits[m])):
            control_sum[i] += int(parts[n][controlled_bits[m][j]])
        control_sum[i] = control_sum[i] % 2
    return control_sum

In [180]:
def full_sums(parts): # здесь все контрольые суммы
    sums = dict()
    for i in range(len(parts)):
        one_sum = count_control_sum(i, parts)
        sums[i] = one_sum
    return sums

In [181]:
reversed_sums = full_sums(reversed_parts)
print(reversed_sums)

{0: [1, 1, 0, 1, 0], 1: [1, 0, 1, 0, 0], 2: [0, 0, 0, 0, 1], 3: [0, 1, 1, 0, 0], 4: [0, 0, 0, 1, 1], 5: [0, 1, 1, 0, 0], 6: [0, 0, 0, 1, 1], 7: [0, 1, 0, 0, 0], 8: [1, 1, 0, 1, 0], 9: [1, 0, 1, 1, 0], 10: [1, 1, 1, 0, 1], 11: [1, 0, 1, 0, 1]}


In [182]:
def get_positions(sums): # превращаем все это в неверные позиции
    positions = dict()
    for i in range(len(sums)):
        positions[i] = str()
        sums[i].reverse()
        for j in range(len(sums[i])):
            positions[i] += (str(sums[i][j]))
    #print(positions)
    return positions

In [183]:
reversed_positions = get_positions(reversed_sums)

In [184]:
def decode_straight_positions(positions): # декодируем в нормальный 10-ый вид
    for i in range(len(positions)):
        positions[i] = int(positions[i],2)
    return positions

In [185]:
decimal_reversed_positions = decode_straight_positions(reversed_positions)
print(decimal_reversed_positions)
for i in range(len(decimal_reversed_positions)):
    if decimal_reversed_positions[i] != 0:
        decimal_reversed_positions[i] -= 1 # позиции по коду Хемминга нумеруются с 1, надо -1 сделать

{0: 11, 1: 5, 2: 16, 3: 6, 4: 24, 5: 6, 6: 24, 7: 2, 8: 11, 9: 13, 10: 23, 11: 21}


In [186]:
def correct_mistakes(positions, parts): # исправляем ошибки
    for i in range(len(parts)):
        if positions[i] < code_word_length:
            b = int(parts[i][positions[i]])
            b = str((b + 1) % 2)
            parts[i] = parts[i][:positions[i]] + b + parts[i][positions[i]+1:]
    return parts
        

In [187]:
print(reversed_parts)

['100110001111010100101010', '101010010011010000010010', '010110011111100001000000', '100010010110011010010011', '001100100011001010011000', '011001111011000011001100', '101100000000101001000101', '100100000110111000111001', '100101101011001010110001', '011100101001000010001101', '010100011000000101111000', '000010000000011011111000']


In [188]:
correct_mistakes(decimal_reversed_positions, reversed_parts)
print(reversed_parts)

['100110001101010100101010', '101000010011010000010010', '010110011111100101000000', '100011010110011010010011', '001100100011001010011001', '011000111011000011001100', '101100000000101001000100', '110100000110111000111001', '100101101001001010110001', '011100101001100010001101', '010100011000000101111010', '000010000000011011110000']


In [189]:
def delete_control_bits(parts): # удаляем контрольные биты
    decoded_strings = list()
    for i in range(len(parts)):
        string = str()
        for j in range(len(parts[i])):
            if j not in bits:
                string += parts[i][j]
        decoded_strings.append(string)
    binary_message = ''.join(decoded_strings)
    return binary_message

In [190]:
binary_reversed_message = delete_control_bits(reversed_parts)
print(binary_reversed_message)
print(len(binary_reversed_message))

010011010100010101010000011010000100100100111110001000000011001100111001001110010011001100110011001101100011001100100000001010100010000000110111001110010011100100110110001100110011001000110100001000000011110100100000001111110000
228


In [191]:
def transform_to_letter_message(binary_message): # постепенно превращаем в бинарное, десятичное и буквенное сообщение
    print(len(binary_message))
    cons = len(binary_message) % 8
    splited_message = []
    for i in range(len(binary_message) // 8):
        splited_message.append(binary_message[(8*i):(8*(i+1))])
    splited_message[-1].zfill(8)
    print(splited_message)
    numbers_message = []
    for i in range(len(splited_message)):
        numbers_message.append(int(splited_message[i],2))
    print(numbers_message)
    letter_message = []
    for i in range(len(splited_message)):
        letter_message.append(chr(numbers_message[i]))
    print(letter_message)
    decoded_message = ''.join(letter_message)
    print(decoded_message)
    return splited_message, numbers_message, letter_message

In [192]:
transform_to_letter_message(binary_reversed_message)

228
['01001101', '01000101', '01010000', '01101000', '01001001', '00111110', '00100000', '00110011', '00111001', '00111001', '00110011', '00110011', '00110110', '00110011', '00100000', '00101010', '00100000', '00110111', '00111001', '00111001', '00110110', '00110011', '00110010', '00110100', '00100000', '00111101', '00100000', '00111111']
[77, 69, 80, 104, 73, 62, 32, 51, 57, 57, 51, 51, 54, 51, 32, 42, 32, 55, 57, 57, 54, 51, 50, 52, 32, 61, 32, 63]
['M', 'E', 'P', 'h', 'I', '>', ' ', '3', '9', '9', '3', '3', '6', '3', ' ', '*', ' ', '7', '9', '9', '6', '3', '2', '4', ' ', '=', ' ', '?']
MEPhI> 3993363 * 7996324 = ?


(['01001101',
  '01000101',
  '01010000',
  '01101000',
  '01001001',
  '00111110',
  '00100000',
  '00110011',
  '00111001',
  '00111001',
  '00110011',
  '00110011',
  '00110110',
  '00110011',
  '00100000',
  '00101010',
  '00100000',
  '00110111',
  '00111001',
  '00111001',
  '00110110',
  '00110011',
  '00110010',
  '00110100',
  '00100000',
  '00111101',
  '00100000',
  '00111111'],
 [77,
  69,
  80,
  104,
  73,
  62,
  32,
  51,
  57,
  57,
  51,
  51,
  54,
  51,
  32,
  42,
  32,
  55,
  57,
  57,
  54,
  51,
  50,
  52,
  32,
  61,
  32,
  63],
 ['M',
  'E',
  'P',
  'h',
  'I',
  '>',
  ' ',
  '3',
  '9',
  '9',
  '3',
  '3',
  '6',
  '3',
  ' ',
  '*',
  ' ',
  '7',
  '9',
  '9',
  '6',
  '3',
  '2',
  '4',
  ' ',
  '=',
  ' ',
  '?'])

In [61]:
l1p2 = 'MEPhI> 3993363 * 7996324 = ?'

In [146]:
answer = '31932224397612'

In [147]:
#answer = '31932224397612'

In [148]:
answer_parts = list(str(answer))
print(answer_parts)

['3', '1', '9', '3', '2', '2', '2', '4', '3', '9', '7', '6', '1', '2']


In [149]:
for i in range(len(answer_parts)):
    answer_parts[i] = bin(ord(answer_parts[i]))[2:].zfill(8)
print(answer_parts)

['00110011', '00110001', '00111001', '00110011', '00110010', '00110010', '00110010', '00110100', '00110011', '00111001', '00110111', '00110110', '00110001', '00110010']


In [150]:
bin_answer = ''.join(answer_parts) # далее делаем +- обратную процедуру
print(bin_answer)

0011001100110001001110010011001100110010001100100011001000110100001100110011100100110111001101100011000100110010


In [151]:
bin_answer_parts = []
cons = len(bin_answer) % (code_word_length - len(bits))
if cons != 0:
    cons = 1
print(cons)
print(len(bin_answer) // (code_word_length - len(bits)))
for i in range(len(bin_answer) // (code_word_length - len(bits)) + cons):
    bin_answer_parts.append(bin_answer[((code_word_length-len(bits))*i):((code_word_length-len(bits))*(i+1))])
#bin_answer_parts[-1] = str(bin(int(bin_answer_parts[-1],2))[2:].zfill(19))
#bin_answer_parts[-1] = str(bin(int(bin_answer_parts[-1],2))[2:]).ljust(19,'0')
bin_answer_parts[-1] = bin_answer_parts[-1].ljust(code_word_length - len(bits), '0')
print(bin_answer_parts)

1
5
['0011001100110001001', '1100100110011001100', '1000110010001100100', '0110100001100110011', '1001001101110011011', '0001100010011001000']


In [152]:
def fulfill(bin_answer_parts): # дозаполняем 0 вместо контрольных битов
    for i in range(len(bin_answer_parts)):
        bin_answer_parts[i] = '00' + bin_answer_parts[i]
        bin_answer_parts[i] = bin_answer_parts[i][:3] + '0' + bin_answer_parts[i][3:6] + '0' + bin_answer_parts[i][6:13] + '0' + bin_answer_parts[i][13:]
    print(bin_answer_parts)
    return bin_answer_parts    

In [153]:
bin_answer_parts = fulfill(bin_answer_parts)
#bin_answer_parts[-1] = bin_answer_parts[-1].ljust(code_word_length, '0')

['000001100011001010001001', '001010001001100011001100', '001000001100100001100100', '000011001000011000110011', '001000100011011010011011', '000000101000100011001000']


In [154]:
control_sums = dict()
control_sums = full_sums(bin_answer_parts) # считаем контрольные суммы
print(control_sums)

{0: [1, 0, 1, 0, 1], 1: [0, 1, 1, 1, 0], 2: [0, 1, 0, 1, 1], 3: [1, 1, 0, 0, 0], 4: [1, 0, 1, 1, 1], 5: [1, 0, 1, 0, 1]}


In [155]:
def put_control_bits(bin_answer_parts, control_sums): # вставляем контрольные биты
    for i in range(len(bin_answer_parts)):
        for j in range(len(bits)):
            n = bits[j]
            if control_sums[i][j] < code_word_length:
                bin_answer_parts[i] = bin_answer_parts[i][:n] + str(control_sums[i][j]) + bin_answer_parts[i][n+1:]
    return bin_answer_parts

In [156]:
bin_answer_parts = put_control_bits(bin_answer_parts, control_sums)
print(bin_answer_parts)

['100101100011001110001001', '011110011001100011001100', '011000011100100101100100', '110011001000011000110011', '101100110011011110011011', '100100101000100111001000']


In [157]:
for i in range(len(bin_answer_parts)):
    bin_answer_parts[i] = bin_answer_parts[i][::-1]
bin_coded_answer = ''.join(bin_answer_parts)
print(bin_coded_answer)

100100011100110001101001001100110001100110011110001001101001001110000110110011000110000100110011110110011110110011001101000100111001000101001001


In [158]:
const = len(bin_coded_answer) % 8
if const != 0:
    const = 1
coded_binary_parts = []
for i in range(len(bin_coded_answer) // 8 + const):
    coded_binary_parts.append(bin_coded_answer[(8*i):(8*(i+1))])
print(coded_binary_parts)
coded_decimal_parts = copy.deepcopy(coded_binary_parts)
for i in range(len(coded_decimal_parts)):
    coded_decimal_parts[i] = int(coded_decimal_parts[i],2)
print(coded_decimal_parts)

['10010001', '11001100', '01101001', '00110011', '00011001', '10011110', '00100110', '10010011', '10000110', '11001100', '01100001', '00110011', '11011001', '11101100', '11001101', '00010011', '10010001', '01001001']
[145, 204, 105, 51, 25, 158, 38, 147, 134, 204, 97, 51, 217, 236, 205, 19, 145, 73]
